In [1]:
import pandas as pd 
data = pd.read_csv('ner_dataset.csv' , encoding = 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [2]:
from itertools import chain 
def get_dict_map(data , token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
        
        
    idx2tok = {idx:tok for idx,tok in enumerate(vocab)}
    tok2idx = {tok:idx for idx,tok in enumerate(vocab)}
    
    return tok2idx , idx2tok

token2idx , idx2token = get_dict_map(data, 'token')
tag2idx , idx2tag = get_dict_map(data ,'tag')

# Transform the column data to train for neural network

In [3]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)

data_fillna = data.fillna(method='ffill' , axis = 0)

#Group by and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

C:/Users/User/AppData/Local/Temp/xpython_288/555980438.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


In [4]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


def get_pad_train_test_val(data_group , data):
    
    #get max and token length 
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))
    
    #Pad token(X Var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s)for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen = maxlen , dtype = 'int32', padding='post' , value = n_token - 1)
    
    #Pad Tags (y var)  and convert it into one encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags , maxlen= maxlen , dtype = 'int32' , padding = 'post' ,value = tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i , num_classes=n_tags) for i in pad_tags]

    #split train test validation set
    token_,test_tokens, tag_ , test_tags =  train_test_split(pad_tokens , pad_tags , test_size =0.1, train_size = 0.9 , random_state = 2020)
    train_tokens , val_tokens,train_tags , val_tags = train_test_split(token_ , tag_ , test_size = 0.25 , train_size =0.75 ,random_state = 2020)
    
    print(
    'train_tokens length : ' , len(train_tokens),
    '\ntrain_tokens length : ' , len(train_tokens),
    '\ntest_tokens length : ' , len(test_tokens),
    '\ntest_tags : ' , len(test_tags),
    '\nval_tokens : ' , len(val_tokens),
    '\nval_tags : ' , len(val_tags)
    )
    
    return train_tokens , val_tokens, test_tokens, train_tags , val_tags , test_tags

train_tokens , val_tokens, test_tokens, train_tags , val_tags , test_tags = get_pad_train_test_val(data_group , data)

train_tokens length :  32372 
train_tokens length :  32372 
test_tokens length :  4796 
test_tags :  4796 
val_tokens :  10791 
val_tags :  10791


# Training Neural Network for Name Entity Recognition

In [5]:
import numpy as np 
import tensorflow 
from tensorflow.keras import Sequential , Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed 
seed(1)
tensorflow.random.set_seed(2)

In [6]:
input_dim = len(list(set(data['Word'].tolist()))) + 1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

# Helper function to get the summary of Neural Network

In [7]:
def get_bilstm_lstm_model():
    
    model = Sequential()
    
    #Add Embedding Layer
    model.add(Embedding(input_dim = input_dim , output_dim = output_dim , input_length = input_length))
    
    #Add Bidirectional LSTM
    model.add(Bidirectional(LSTM(units = output_dim,return_sequences = True , dropout = 0.2 , recurrent_dropout = 0.2) ,merge_mode = 'concat'))
    
    #Add LSTM
    model.add(LSTM(units = output_dim , return_sequences = True , dropout = 0.5 , recurrent_dropout = 0.5))
    
    #Add Timedistributed layer 
    model.add(TimeDistributed(Dense(n_tags, activation = 'relu')))
    
    #Complie model 
    model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' ,metrics = ['accuracy'])
    model.summary()
    
    return model

In [8]:
def train_model(X,y, model):
    loss = list()
    for i in range(25):
        #fit model for one epoch on this sequence
        hist = model.fit(X,y, batch_size = 1000, verbose = 1, epochs =1, validation_split = 0.2)
        loss.append(hist.history['loss'][0])
        
    return loss

In [9]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags) , model_bilstm_lstm)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 104, 64)           2251456   
_________________________________________________________________
bidirectional (Bidirectional (None, 104, 128)          66048     
_________________________________________________________________
lstm_1 (LSTM)                (None, 104, 64)           49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 104, 17)           1105      
Total params: 2,368,017
Trainable params: 2,368,017
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')

26/26 [==============================] - 260s 10s/step - loss: 0.4888 - accuracy: 0.9596 - val_

26/26 [==============================] - 383s 15s/step - loss: 0.2096 - accuracy: 0.9677 - val_loss: 0.1869 - val_accuracy: 0.9681

26/26 [==============================] - 415s 16s/step - loss: 0.1994 - accuracy: 0.9677 - val_loss: 0.1974 - val_accuracy: 0.9681

26/26 [==============================] - 398s 15s/step - loss: 0.3675 - accuracy: 0.9652 - val_loss: 0.2681 - val_accuracy: 0.9680

14/26 [===============>..............] - ETA: 3:02 - loss: 0.2355 - accuracy: 0.9675


26/26 [==============================] - 426s 16s/step - loss: 0.2526 - accuracy: 0.9676 - val_loss: 0.2395 - val_accuracy: 0.9681

26/26 [==============================] - 420s 16s/step - loss: 0.1957 - accuracy: 0.9677 - val_loss: 0.1703 - val_accuracy: 0.9681

26/26 [==============================] - 428s 16s/step - loss: 0.1592 - accuracy: 0.9677 - val_loss: 0.1465 - val_accuracy: 0.9681

26/26 [==============================] - 427s 16s/step - loss: 0.1425 - accuracy: 0.9677 - val_loss: 0.1352 - val_accuracy: 0.9681



26/26 [==============================] - 442s 17s/step - loss: 0.1428 - accuracy: 0.9678 - val_loss: 0.1354 - val_accuracy: 0.9681

26/26 [==============================] - 449s 17s/step - loss: 0.1265 - accuracy: 0.9678 - val_loss: 0.1280 - val_accuracy: 0.9681

26/26 [==============================] - 461s 18s/step - loss: 0.1235 - accuracy: 0.9678 - val_loss: 0.1231 - val_accuracy: 0.9682

13/26 [==============>...............] - ETA: 3:31 - loss: 0.1181 - accuracy: 0.9678


26/26 [==============================] - 466s 18s/step - loss: 0.1164 - accuracy: 0.9678 - val_loss: 0.1189 - val_accuracy: 0.9682

26/26 [==============================] - 470s 18s/step - loss: 0.1117 - accuracy: 0.9679 - val_loss: 0.1183 - val_accuracy: 0.9682

26/26 [==============================] - 491s 19s/step - loss: 0.1073 - accuracy: 0.9680 - val_loss: 0.1145 - val_accuracy: 0.9683

26/26 [==============================] - 526s 20s/step - loss: 0.1193 - accuracy: 0.9683 - val_loss: 0.1207 - val_accuracy: 0.9685



26/26 [==============================] - 555s 21s/step - loss: 0.1593 - accuracy: 0.9681 - val_loss: 0.1940 - val_accuracy: 0.9686

26/26 [==============================] - 569s 22s/step - loss: 0.1622 - accuracy: 0.9682 - val_loss: 0.1298 - val_accuracy: 0.9686

26/26 [==============================] - 465s 18s/step - loss: 0.1165 - accuracy: 0.9681 - val_loss: 0.1148 - val_accuracy: 0.9685

13/26 [==============>...............] - ETA: 3:22 - loss: 0.1100 - accuracy: 0.9683


26/26 [==============================] - 441s 17s/step - loss: 0.1082 - accuracy: 0.9683 - val_loss: 0.1111 - val_accuracy: 0.9686

26/26 [==============================] - 447s 17s/step - loss: 0.2015 - accuracy: 0.9665 - val_loss: 0.1866 - val_accuracy: 0.9691

26/26 [==============================] - 446s 17s/step - loss: 0.1502 - accuracy: 0.9684 - val_loss: 0.1267 - val_accuracy: 0.9687

26/26 [==============================] - 451s 17s/step - loss: 0.1106 - accuracy: 0.9683 - val_loss: 0.1133 - val_accuracy: 0.9687



26/26 [==============================] - 452s 17s/step - loss: 0.1009 - accuracy: 0.9684 - val_loss: 0.1066 - val_accuracy: 0.9688


# Testing the Name Entity Recognition Model

In [10]:
import spacy 
from spacy import displacy 
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Anshul Mahajan \n I am from Indore, Madhya Pradesh \n I look up to Elon musk as my inspiration')

displacy.render(text , style = 'ent' , jupyter = True)